In [1]:
%matplotlib notebook
import pandas as pd
import numpy as np
import scipy as sp
import plotly.plotly as py
import plotly.graph_objs as go
import matplotlib.pyplot as plt

In [2]:
import pandas as pd
import numpy as np
import PIL.Image
import PIL
from ipywidgets import Image

import phate
import io
import sys
sys.path.append("..")
from blog_tools import data

dataset = data.digits()

In [3]:
def image_to_bytes(im, mode="L"):
    im = PIL.Image.fromarray(im, mode="L")
    imgByteArr = io.BytesIO()
    im.save(imgByteArr, format='PNG')
    imgByteArr = imgByteArr.getvalue()
    return imgByteArr

In [4]:
Y = phate.PHATE().fit_transform(dataset.X)
fig = go.FigureWidget(
    data=[
        dict(
            type='scattergl',
            x=Y[:,0],
            y=Y[:,1],
            mode='markers',
        )
    ],
)

Calculating PHATE...
  Running PHATE on 179 cells and 64 genes.
  Calculating graph and diffusion operator...
    Calculating KNN search...
    Calculating affinities...
  Calculated graph and diffusion operator in 0.01 seconds.
  Calculating optimal t...
    Automatically selected t = 23
  Calculated optimal t in 0.01 seconds.
  Calculating diffusion potential...
  Calculating metric MDS...
  Calculated metric MDS in 0.21 seconds.
Calculated PHATE in 0.24 seconds.


In [5]:
fig.layout.title = dataset.name
fig.layout.titlefont.size
fig.layout.titlefont.size = 22
fig.layout.titlefont.family = 'Rockwell'
fig.layout.xaxis.title = 'PHATE1'
fig.layout.yaxis.title = 'PHATE2'

In [6]:
scatter = fig.data[0]
scatter.marker.opacity = 0.7
scatter.marker.size = 10
fig.layout.hovermode = 'closest'

In [7]:
contour = fig.add_histogram2dcontour(
    x=scatter.x, y=scatter.y, contours={'coloring':'lines'})
contour.colorscale = [[i, "rgb({}, {}, {})".format(round(r*255), round(g*255), round(b*255))]
                        for i, (r,g,b) in zip(
                            np.linspace(0, 1, len(plt.cm.inferno.colors)), 
                            plt.cm.inferno.colors)]
contour.reversescale = True
contour.hoverinfo = 'skip'

In [8]:
def set_opacity(opacity, size):
    scatter.marker.opacity = opacity
    scatter.marker.size = size
    
from ipywidgets import interactive
opacity_slider = interactive(set_opacity,
                             opacity=(0.0, 1.0, 0.01),
                             size=(1, 10, 0.25))
opacity_slider

interactive(children=(FloatSlider(value=0.5, description='opacity', max=1.0, step=0.01), FloatSlider(value=5.0…

In [9]:
opacity_slider.children[0].layout.width = '400px'
opacity_slider.children[1].layout.width = '400px'

In [10]:
from ipywidgets import Image, Layout
image_widget = Image(
    value=image_to_bytes(dataset.X_raw[0]),
    layout=Layout(height='252px', width='252px')
)
display(image_widget)

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00\x08\x00\x00\x00\x08\x08\x00\x00\x00\x00\xe1d\xe1…

In [12]:
def hover_fn(trace, points, state):

    ind = points.point_inds[0]
    
    # Update image widget
    image_widget.value = image_to_bytes(dataset.X_raw[ind])

scatter.on_hover(hover_fn)

In [13]:
from ipywidgets import HBox, VBox
dash = VBox([fig,
      HBox([opacity_slider, image_widget])])
display(dash)

    'data': [{'marker': {'opacity': 0.5, 'size': 5.0},
              'mode': 'ma…